<a href="https://colab.research.google.com/github/RMoulla/NLP/blob/main/Topic_Modeling_Correction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Topic Modeling sur des avis de consommateurs

Dans ce tutoriel, nous allons utiliser une technique de topic modeling pour extraire des thématiques significatives à partir de données textuelles. Le jeu de données choisi est composé d'avis de clients sur des produits électroniques vendus sur Amazon. Ce dataset comprend des informations telles que le texte de l'avis, la note globale attribuée au produit, un résumé de l'avis, ainsi que des informations temporelles liées à chaque avis.

La technique utilisée ici est Latent Dirichlet Allocation (LDA), un modèle statistique adapté pour l'identification de thématiques latentes dans de vastes ensembles de données textuelles.

LDA est un modèle génératif probabiliste qui permet de découvrir des groupes de mots (appelés "topics" ou thématiques) dans un corpus de documents. Il est basé sur deux hypothèses principales :

* Chaque document est un mélange de sujets : LDA suppose que chaque document dans notre ensemble de données peut être décrit par un certain nombre de sujets différents. Par exemple, un avis sur un smartphone pourrait inclure des sujets tels que la performance de la batterie, la qualité de l'appareil photo, ou l'expérience utilisateur.

* Chaque sujet est un mélange de mots : Chaque sujet identifié par LDA est caractérisé par un ensemble de mots. Par exemple, le sujet "performance de la batterie" pourrait inclure des mots comme "longue durée", "recharge", "autonomie", etc.


In [ ]:
!pip install pyldavis==3.4.1
!pip install pandas==1.5.1

  Using cached pandas-2.1.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.3 MB)
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.1
    Uninstalling pandas-1.5.1:
      Successfully uninstalled pandas-1.5.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
bigframes 0.18.0 requires pandas<2.1.4,>=1.5.0, but you have pandas 2.1.4 which is incompatible.
google-colab 1.0.0 requires pandas==1.5.3, but you have pandas 2.1.4 which is incompatible.
  Using cached pandas-1.5.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.1 MB)
  Attempting uninstall: pandas
    Found existing installation: p

## Importation des librairies nécessaires

Ce code permet d'importer l'ensemble des bibliothèques nécessaires pour le script. **pandas** est utilisé pour le chargement et la manipulation des données, **re** pour les expressions régulières (pour le prétraitement du texte), **sklearn** pour le topic modeling, **nltk** pour le traitement du langage naturel et **pyLDAvis** pour la visualisation.

In [ ]:
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import nltk
import pyLDAvis
import pyLDAvis.lda_model

Nous téléchargeons ensuite les ressources NLTK nécessaires : **wordnet** pour la lemmatisation ainsi que la liste prédéfinie des stopwords.

In [ ]:
# Téléchargement des ressources NLTK pour le prétraitement du texte
nltk.download('wordnet')
nltk.download('stopwords')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

## Prétraitement du texte

Nous initalisaons le lemmatizer et la liste des stopwords et nous définissons une fonction **preprocess_text** pour le prétraitement des avis. Cette fonction nettoie le texte, le convertit en minuscules, supprime les stopwords, et applique la lemmatisation.

In [ ]:
# Initialisation du lemmatizer et des stop words
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

# Fonction de prétraitement du texte
def preprocess_text(text):
    text = re.sub(r'[^a-zA-Z]', ' ', text.lower())
    words = [lemmatizer.lemmatize(word) for word in text.split() if word not in stop_words]
    return ' '.join(words)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Chargement des données

Ici, nous chargeons les données à partir d'un fichier CSV, nettoyons les données en supprimant les lignes sans avis, puis appliquons la fonction de prétraitement à chaque avis.

In [ ]:
# Charger les données
data = pd.read_csv("electronics_sample.csv")
data = data.dropna(subset=['reviewText'])
text_data = data['reviewText'].values

# Application du prétraitement
preprocessed_text = [preprocess_text(text) for text in text_data]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Vectorisation et modélisation

Avant d'effectuer la modélisation, les mots sont transformés en valeurs numériques (vecteurs) à l'aide de **CountVectorizer**. Celui-ci peut considérer ausi bien des 1-gram que des bigrams, etc. Les mots sont ensuite filtrés à l'aide de **max_df** et **min_df** pour ignorer les mots qui sont trop fréquents ou trop rares. Cela permet à l'algorithme LDA de se concentrer sur les mots significatifs et réduit la dimensionnalité du dataset. Le modèle est entraîné sur la matrice document-terme formée.


In [ ]:
# Vectorisation du texte avec bigrammes
vectorizer = CountVectorizer(max_df=0.95, min_df=2, stop_words='english', ngram_range=(1, 2))
dtm = vectorizer.fit_transform(preprocessed_text)

# Création et ajustement du modèle LDA
lda = LatentDirichletAllocation(n_components=5, random_state=0, n_jobs=1)
lda.fit(dtm)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


LatentDirichletAllocation(n_components=5, n_jobs=1, random_state=0)

 ## Visualisation des thématiques

 Ce code prépare les données pour **pyLDAvis** et renvoie une visualisation interactive des sujets trouvés dans les avis. Cela permet une exploration visuelle et intuitive des thématiques et de leurs mots clés associés.

In [ ]:
# Préparation des données pour pyLDAvis
pyLDAvis.enable_notebook()
lda_display = pyLDAvis.lda_model.prepare(lda, dtm, vectorizer, sort_topics=False)

# Affichage de la visualisation
pyLDAvis.display(lda_display)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/pandas/core/dtypes/cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])
